# Quantum Convolutional Processing of the Dataset

This notebook contains all the rutines to apply quantum convolution to an entire dataset. The script will:

1. Load the dataset and print some statistics
2. Create a quantum cicuit
3. Define a quantum convolutional layer
4. Apply the quantum convolutional layer to the entire dataset and save a new version of the dataset

In [ ]:
from data.datahandler import datahandler
from data.datareader import datareader
from layers.QConv2D import QConv2D
from utils import test_loader
from utils.plotter import *

import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pennylane as qml
import numpy as np
import os

import jax; jax.config.update('jax_platform_name', 'cpu')

import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Define Quantum Circuit 

The quantum circuit is used by the quantum convolutional 2D layer. The quantum circuit used in this example has a fixed structure, Ry gates at the beginning, Random Layers U and Z pauli and measurement gates at the end.

```
                                                   _____
                                    |0> ---[Ry]---|     |---[Z]---[m]
                                    |0> ---[Ry]---|     |---[Z]---[m]
                                    |0> ---[Ry]---|  U  |---[Z]---[m]
                                    |0> ---[Ry]---|     |
                                    |0> ..........|_____|
              
```

It is defined by 6 parameters:

1. *QUBITS*: is the number of qubits used, the number is related to the kernel size of convolution. The relation between these two variables is the following:
$$QUBITS \geq KERNEL\_SIZE^2$$

2. *KERNEL_SIZE*: defines the size of the kernel and the number of RY gates
3. *FILTERS*: defines the number of filters produced by the Quantum Convolution
4. *N_LAYERS*: defines the number of U random layers
5. *STRIDE*: defines the stride used for the convolution
6. *NUM_JOBS*: defines the number of workers for Quantum Convolutional parallelization

In [ ]:
QUBITS      = 16
KERNEL_SIZE = 2
FILTERS     = 16
N_LAYERS    = 1
STRIDE      = 1
NUM_JOBS    = 16
SEED        = 1

## Quantum Convolutional 2D/3D layer

The quantum convolutional 2D/3D layer, uses the quantum circuit define above. As for classical convolution a chuck of the input image of kernel_size dimensions is processed by the quantum circuit that then produces the feature maps. Then as for the classical convolution stride by stride all the chucks of the image are processed.

```
_____________________________________________ Iteration 1 _____________________________________________

                                                                                         Output
                                                                                     ______________
                                                                                    | m3 |    |    |
                                                                                  ______________  _|
        Input Image                                                              | m2 |    |    |  |
    ___________________                                                        ______________  _| _|
   | x1 | x3 |    |    |                                                    __| m1 |    |    |  |  |
   |_|__|_|__|____|____|                                                   |  |____|____|____| _| _|
   | x2 | x4 |    |    |                                                   |  |    |    |    |  |
   |_|__|_|__|____|____|                                                   |  |____|____|____| _|
   | |  | |  |    |    |                                                   |  |    |    |    | 
   |_|__|_|__|____|____|                                                   |  |____|____|____|                     
   | |  | |  |    |    |                                                   | 
   |_|__|_|__|____|____|                                                   | 
     |    |                               Quantum Circuit                  |
     |____|_________________________                            ___________|
                                    |         _____            |
                           |0> ---[Ry(x1)]---|     |---[Z]---[m1]
                           |0> ---[Ry(x2)]---|     |---[Z]---[m2]
                           |0> ---[Ry(x3)]---|  U  |---[Z]---[m3]
                           |0> ---[Ry(x4)]---|_____|


_____________________________________________ Iteration 2 _____________________________________________

                                                                                         Output
                                                                                     ______________
                                                                                    | m3 | m6 |    |
                                                                                  ______________  _|
        Input Image                                                              | m2 | m5 |    |  |
    ___________________                                                        ______________  _| _|
   |    | x3 | x5 |    |                                                    __| m1 | m4 |    |  |  |
   |____|_|__|_|__|____|                                                   |  |____|____|____| _| _|
   |    | x4 | x6 |    |                                                   |  |    |    |    |  |
   |____|_|__|_|__|____|                                                   |  |____|____|____| _|
   |    | |  | |  |    |                                                   |  |    |    |    | 
   |____|_|__|_|__|____|                                                   |  |____|____|____|                     
   |    | |  | |  |    |                                                   | 
   |____|_|__|_|__|____|                                                   | 
          |    |                          Quantum Circuit                  |
          |____|____________________                            ___________|
                                    |         _____            |
                           |0> ---[Ry(x3)]---|     |---[Z]---[m4]
                           |0> ---[Ry(x4)]---|     |---[Z]---[m5]
                           |0> ---[Ry(x5)]---|  U  |---[Z]---[m6]
                           |0> ---[Ry(x6)]---|_____|
              


```

In [ ]:
circuit = 'rxyz' # or rx or rz or rxyz

conv1 = QConv2D(
    QUBITS,
    FILTERS, 
    KERNEL_SIZE, 
    STRIDE, 
    parallelize = NUM_JOBS,
    circuit = circuit
)

## Load image

In [ ]:
#path='datasets/test/s1.png'
path = '/Users/asebastianelli/Desktop/quanvolutional4eo/datasets/EuroSAT/River/River_1007.jpg'
img1 = plt.imread(path)[:,:,:3]
if img1.max() > 100: img1 = img1/255.0

In [ ]:
img1.shape

In [ ]:
out1 = conv1.apply(img1, verbose = False)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = ((FILTERS+1)*11,10))

img = img1.copy()
for i in range(FILTERS):
    c = out1[:,:,i]
    c = (c+1)*0.5
    a = np.pad(c, ((0, img1.shape[0]-c.shape[0]), (0, img1.shape[1]-c.shape[1])), 'constant', constant_values=0) #np.pad(c, 2)#int(1+np.abs(img1.shape[0]-out1.shape[0])//2))
    b = np.repeat(a[:,:,None], 3, axis = 2)

    img = np.concatenate((img, np.ones((img1.shape[0], 1, 3)), b), axis = 1)
    

ax.imshow(img, vmin = 0, vmax = 1)
ax.axis(False)
fig.tight_layout()
#plt.savefig('results/q{}-k{}-s{}.png'.format(QUBITS, KERNEL_SIZE,STRIDE))